# Optimize Baseline Pulses (HB-GRAPE lite via sesolve)

Loads the GRAPE baseline pulses, parameterizes additive harmonic bases (8 for Omega, 4 for Delta), and optimizes the coefficients to maximize terminal population in |1> using QuTiP `sesolve`.

Outputs overwrite the CMA-ES originals folder for consistency:
- notebooks/results/CMA-ES/Omega_Original.npy
- notebooks/results/CMA-ES/Delta_Original.npy
Also saves the loaded baselines as `Omega_Original_base.npy` and `Delta_Original_base.npy` in the same folder.


In [ ]:
import numpy as np
from pathlib import Path
from dataclasses import dataclass
from typing import Tuple
import qutip as qt
from scipy.optimize import minimize

# Keep prints compact
np.set_printoptions(precision=4, suppress=True)

In [ ]:
BASE_DIR = Path('/home/yehon/projects/grape-crab-qoc')
GRAPE_DIR = BASE_DIR / 'notebooks' / 'results' / 'GRAPE'
OUT_DIR = BASE_DIR / 'notebooks' / 'results' / 'CMA-ES'

OMEGA_BASE_PATH = GRAPE_DIR / 'OMEGA_baseline.npy'
DELTA_BASE_PATH = GRAPE_DIR / 'DELTA_baseline.npy'

# Time grid — inferred from baseline length over 0.1 us window
DURATION = 0.1


In [ ]:
omega_base = np.load(OMEGA_BASE_PATH)
delta_base = np.load(DELTA_BASE_PATH)
assert omega_base.ndim == 1 and delta_base.ndim == 1, 'Expected 1D baseline pulses.'
assert omega_base.shape == delta_base.shape, 'Omega/Delta baseline lengths must match.'
N = omega_base.size
T = DURATION

# Canonical tlist
T_LIST = np.linspace(0.0, T, N)
print(f'Loaded baselines: N={N}, T={T} (s)')

# Save baseline copies for provenance
OUT_DIR.mkdir(parents=True, exist_ok=True)
np.save(OUT_DIR / 'Omega_Original_base.npy', omega_base)
np.save(OUT_DIR / 'Delta_Original_base.npy', delta_base)
print('Saved baseline copies.')


In [ ]:
def fourier_basis_matrix(t: np.ndarray, K: int) -> np.ndarray:
    # Columns: [sin(1), cos(1), sin(2), cos(2), ..., sin(K), cos(K)]
    mat = []
    for k in range(1, K + 1):
        w = 2 * np.pi * k / t[-1] if t[-1] != 0 else 0.0
        mat.append(np.sin(w * t))
        mat.append(np.cos(w * t))
    return np.column_stack(mat) if mat else np.zeros((t.size, 0))

K_OMEGA = 8
K_DELTA = 4

PHI_OMEGA = fourier_basis_matrix(T_LIST, K_OMEGA)
PHI_DELTA = fourier_basis_matrix(T_LIST, K_DELTA)

n_omega = PHI_OMEGA.shape[1]
n_delta = PHI_DELTA.shape[1]
print(f'Basis dims: omega={n_omega}, delta={n_delta}')


In [ ]:
sx = qt.sigmax(); sz = qt.sigmaz()
ket0 = qt.basis(2, 0)
proj1 = qt.basis(2, 1) * qt.basis(2, 1).dag()


def coeff_from_values(values: np.ndarray):
    values = np.asarray(values, dtype=float)
    def f(t, _=None):
        return np.interp(t, T_LIST, values)
    return f


def build_controls(theta: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    # theta = [theta_omega, theta_delta]
    theta_omega = theta[:n_omega]
    theta_delta = theta[n_omega:]
    omega = omega_base + PHI_OMEGA @ theta_omega
    delta = delta_base + PHI_DELTA @ theta_delta
    return omega, delta


def run_dynamics(omega: np.ndarray, delta: np.ndarray) -> float:
    H = [
        [0.5 * sx, coeff_from_values(omega)],
        [0.5 * sz, coeff_from_values(delta)],
    ]
    res = qt.sesolve(H, ket0, T_LIST, e_ops=[proj1])
    return float(np.real(res.expect[0][-1]))  # final |1> population


def objective(theta: np.ndarray) -> float:
    omega, delta = build_controls(theta)
    p1 = run_dynamics(omega, delta)
    # Minimize cost = 1 - P1(T)
    return 1.0 - p1


In [ ]:
# Initial guess: zeros for additive harmonics
x0 = np.zeros(n_omega + n_delta, dtype=float)

# Box constraints (optional). Keep additive terms moderate to avoid blowups.
# Here, allow each harmonic coefficient in [-2, 2].
bounds = [(-2.0, 2.0)] * x0.size

result = minimize(
    objective,
    x0,
    method='L-BFGS-B',
    bounds=bounds,
    options=dict(maxiter=200, ftol=1e-6, maxfun=5000, disp=True),
)

print('Optimization success:', result.success)
print('Final cost:', result.fun)


In [ ]:
theta_opt = result.x
omega_opt, delta_opt = build_controls(theta_opt)

np.save(OUT_DIR / 'Omega_Original.npy', omega_opt)
np.save(OUT_DIR / 'Delta_Original.npy', delta_opt)
print('Saved optimized pulses:')
print(' -', OUT_DIR / 'Omega_Original.npy')
print(' -', OUT_DIR / 'Delta_Original.npy')

# Quick before/after terminal populations
p1_base = run_dynamics(omega_base, delta_base)
p1_opt = run_dynamics(omega_opt, delta_opt)
print(f'Final population |1>: baseline={p1_base:.6f}, optimized={p1_opt:.6f}')
